У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [38]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE, SMOTENC
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.multiclass import OneVsRestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline

In [39]:
raw_df = pd.read_csv('customer_segmentation_train.csv')

In [40]:
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [41]:
train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=42, stratify=raw_df['Segmentation'])

input_cols = train_df.columns[1:-1]  # to drop 'ID' and 'Segmentation' columns

# Створюємо трен. і вал. набори
input_cols = train_df.columns[1:-1]
target_col = 'Segmentation'
train_inputs, train_targets = train_df[input_cols], train_df[target_col]
test_inputs, test_targets = test_df[input_cols], test_df[target_col]

In [42]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes(include='object').columns.tolist()
binary_cols = [col for col in categorical_cols if train_inputs[col].nunique() == 2]
categorical_cols = [col for col in categorical_cols if col not in binary_cols]

print('numeric:', numeric_cols)
print('binary_cols:', binary_cols)
print('categorical_cols:', categorical_cols)

numeric: ['Age', 'Work_Experience', 'Family_Size']
binary_cols: ['Gender', 'Ever_Married', 'Graduated']
categorical_cols: ['Profession', 'Spending_Score', 'Var_1']


In [43]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('bin', binary_transformer, binary_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

X_train = preprocessor.fit_transform(train_inputs)
y_train = train_targets

X_test = preprocessor.transform(test_inputs)
y_test = test_targets

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [44]:
X_numeric = numeric_transformer.fit_transform(train_inputs[numeric_cols])

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_numeric, train_targets)

X_test_smote = numeric_transformer.transform(test_inputs[numeric_cols])

original_counts = train_targets.value_counts()
resampled_counts = pd.Series(y_train_smote).value_counts()

print('Original:')
for value, count in sorted(original_counts.items()):
    print(f'{value}- {count}')
print('Resampled:')
for value, count in resampled_counts.items():
    print(f'{value}- {count}')

Original:
A- 1578
B- 1486
C- 1576
D- 1814
Resampled:
A- 1814
B- 1814
C- 1814
D- 1814


In [45]:
cols = categorical_cols + binary_cols
categorical_features_indices = [train_inputs.columns.get_loc(col) for col in cols]

smotenc = SMOTENC(categorical_features=categorical_features_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

original_counts = train_targets.value_counts()
resampled_counts = pd.Series(y_train_smotenc).value_counts()

print('Original:')
for value, count in sorted(original_counts.items()):
    print(f'{value}- {count}')
print('Resampled:')
for value, count in resampled_counts.items():
    print(f'{value}- {count}')

Original:
A- 1578
B- 1486
C- 1576
D- 1814
Resampled:
A- 1814
B- 1814
C- 1814
D- 1814


In [46]:
smote_tomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smote_tomek.fit_resample(X_train, y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [47]:
log_reg = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
ovr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', OneVsRestClassifier(log_reg))
])
ovr_model.fit(train_inputs, train_targets)
ovr_predictions = ovr_model.predict(test_inputs)

print(classification_report(y_test, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.42      0.17      0.24       372
           C       0.49      0.63      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.48      1614
weighted avg       0.50      0.52      0.50      1614



In [48]:
# SMOTENC

pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTENC(categorical_features=categorical_features_indices, random_state=42)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(test_inputs)
print(classification_report(y_test, ovr_predictions))     

              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.40      0.24      0.30       372
           C       0.50      0.59      0.54       394
           D       0.68      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [49]:
# SMOTE-Tomek

pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote_tomek', SMOTETomek(random_state=42)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(test_inputs)
print(classification_report(y_test, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.49      0.45       394
           B       0.42      0.25      0.31       372
           C       0.50      0.60      0.54       394
           D       0.69      0.70      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



**Висновок:**


Для порівняння використав методику f1-score.

Модель SMOTENC та SMOTE-Tomek показали кращий та досить схожий результат.

На мою думку немає суттєвої різниці між моделями, тому що початкові дані досить збалансовані.